### Imports

In [ ]:
from pyspark.sql import SparkSession
import os
import sys
# import pandas as pd
# import pyspark

dir_path = os.getcwd()
root_path = os.path.abspath(os.path.join(dir_path, '..'))
sys.path.append(root_path)

from migration_library.main import tbl_parsed_dict, relation_fct_dict, relation_dim_dict, tbl_name_reformat

In [ ]:
import secret_scope

### ADLS Connection

In [ ]:
storage_account = secret_scope.storage_account
container_name = secret_scope.container_name
path = secret_scope.path

In [ ]:
spark = SparkSession.builder \
    .appName("sparkwriteapp") \
    .getOrCreate()

# Configure direct access via abfss
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "ManagedIdentity")         # Assuming IAM is setup and configured
spark.conf.set("fs.azure.account.oauth.msi.endpoint", "http://169.254.169.254/metadata/identity/oauth2/token")

### Write to ext location

In [ ]:
# External table write location
adls_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{path}"

# Catalog and schema information | Can be parameterised in actual usecase
catalog_name = 'data_foundation_dev'
schema_name = 'raw'

In [ ]:
for tbl, data in tbl_parsed_dict:
    df = data['df']
    tbl_name = tbl_name_reformat(tbl)

    spark_df = spark.createDataFrame(df)
    storage_path = f"{adls_path}/{schema_name}/{tbl_name}"

    spark_df.write.format(format) \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .option("path", adls_path) \
            .saveAsTable(f"{catalog_name}.{schema_name}.{tbl_name}")

    print(f"External table `{catalog_name}`.{schema_name}.{tbl_name} created successfully with data at {storage_path}.")